In [90]:
from torchvision import models
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image

In [91]:
# Load the model from a file
alexnet = torch.load("alexnet-pretrained.pt")
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [92]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

In [93]:
# Separate TEST new submodels
modules = list(alexnet.children())[:1]
Main_Submodel1 = nn.Sequential(*modules)
                                  

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


In [94]:
# iterate over children leaf node
for n in Main_Submodel1.children():
  submodel_1 = nn.Sequential(n[:1])
  submodel_2 = nn.Sequential(n[1:2])
  submodel_3 = nn.Sequential(n[2:3])
  submodel_4 = nn.Sequential(n[3:4])
  submodel_5 = nn.Sequential(n[4:5])
  submodel_6 = nn.Sequential(n[5:6])
  submodel_7 = nn.Sequential(n[6:7])
  submodel_8 = nn.Sequential(n[7:8])
  submodel_9 = nn.Sequential(n[8:9])
  submodel_10 = nn.Sequential(n[9:10])
  submodel_11 = nn.Sequential(n[10:])

In [95]:
avgpoolLayer = list(alexnet.children())[1]
submodel_avg = nn.Sequential(avgpoolLayer)
print(submodel_avg)

Sequential(
  (0): AdaptiveAvgPool2d(output_size=(6, 6))
)


In [96]:
# create submodels from testsubmodel2 children layers
modules = list(alexnet.children())[2:]
Main_Submodel2 = nn.Sequential(submodel_avg, Flatten(), *modules)
print(Main_Submodel2)

Sequential(
  (0): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(6, 6))
  )
  (1): Flatten()
  (2): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=4096, out_features=4096, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)


In [97]:
# flatten layer
submodel_flatten = nn.Sequential(Flatten())
print(submodel_flatten)

Sequential(
  (0): Flatten()
)


In [98]:
# Prepare a transform to get the input image into a format (e.g., x,y dimensions) the classifier
# expects
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])

img = Image.open("input.jpg")

# Apply the transform to the image.
img_t = transform(img)

# Magic (not sure what this does).
batch_t = torch.unsqueeze(img_t, 0)

In [99]:
torch.save(submodel_1, "submodel_1.pt")
torch.save(submodel_2, "submodel_2.pt")
torch.save(submodel_3, "submodel_3.pt")
torch.save(submodel_4, "submodel_4.pt")
torch.save(submodel_5, "submodel_5.pt")
torch.save(submodel_6, "submodel_6.pt")
torch.save(submodel_7, "submodel_7.pt")
torch.save(submodel_8, "submodel_8.pt")
torch.save(submodel_9, "submodel_9.pt")
torch.save(submodel_10, "submodel_10.pt")
torch.save(submodel_11, "submodel_11.pt")
torch.save(Main_Submodel2, "Main_Submodel2.pt")

In [100]:
submodel_1.eval()
submodel_2.eval()
submodel_3.eval()
submodel_4.eval()
submodel_5.eval()
submodel_6.eval()
submodel_7.eval()
submodel_8.eval()
submodel_9.eval()
submodel_10.eval()
submodel_11.eval()
Main_Submodel2.eval()

Sequential(
  (0): Sequential(
    (0): AdaptiveAvgPool2d(output_size=(6, 6))
  )
  (1): Flatten()
  (2): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=4096, out_features=4096, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)

In [101]:
output_submodel_1 = submodel_1(batch_t)
output_submodel_2 = submodel_2(output_submodel_1)
output_submodel_3 = submodel_3(output_submodel_2)
output_submodel_4 = submodel_4(output_submodel_3)
output_submodel_5 = submodel_5(output_submodel_4)
output_submodel_6 = submodel_6(output_submodel_5)
output_submodel_7 = submodel_7(output_submodel_6)
output_submodel_8 = submodel_8(output_submodel_7)
output_submodel_9 = submodel_9(output_submodel_8)
output_submodel_10 = submodel_10(output_submodel_9)
output_submodel_11 = submodel_11(output_submodel_10)
outputFinal = Main_Submodel2(output_submodel_11)

In [102]:
# Load the classes from disk.
with open('classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

# Sort the predictions.
_, indices = torch.sort(outputFinal, descending=True)

# Convert into percentages.
percentage = torch.nn.functional.softmax(outputFinal, dim=1)[0] * 100

# Print the 5 most likely predictions.
with open("result_Alexnet_12.txt", "w") as outfile:
    outfile.write(str([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]))

print(open('result_Alexnet_12.txt', 'r').read()) 

[('Labrador retriever', 41.58519744873047), ('golden retriever', 16.591642379760742), ('Saluki, gazelle hound', 16.28689193725586), ('whippet', 2.853909969329834), ('Ibizan hound, Ibizan Podenco', 2.3924717903137207)]
